In [ ]:
%%configure -f
{
    "conf": {
        "spark.jars": "abfss://<container>@<storage_account_name>.dfs.core.windows.net/jars/delta-core_2.12-2.1.1.jar"
    }
}

In [ ]:
# build lakebench zip and upload to ADLS Gen2
sc.addPyFile("abfss://<container>@<storage_account_name>.dfs.core.windows.net/libs/lakebench.zip")
import lakebench

In [ ]:
# Enable arbitrary Delta table properties to prevent failure if LakeBench attempts to set newer properties that are not the HDI compatible version of Delta Lake
spark.conf.set('spark.databricks.delta.allowArbitraryProperties.enabled', True)

## Run ELTBench in `light` mode

In [ ]:
from lakebench.engines import HDISpark
from lakebench.benchmarks import ELTBench

engine = HDISpark(
    schema_name ='spark_eltbench_test',
    spark_measure_telemetry = False
)

benchmark = ELTBench(
    engine=engine,
    scenario_name="SF1",
    input_parquet_folder_uri='abfss://........./Files/tpcds_sf1',
    save_results=True,
    result_table_uri='abfss://......../Tables/lakebench/results'
    )
benchmark.run(mode="light")

## Run TPCDS `power_test` (Load tables and run all queries)

In [ ]:
from lakebench.engines import HDISpark
from lakebench.benchmarks import TPCDS

engine = HDISpark(
    schema_name = 'spark_tpcds_sf1',
    spark_measure_telemetry = False
)

benchmark = TPCDS(
    engine=engine,
    scenario_name="SF1 - Power Test",
    input_parquet_folder_uri='abfss://........./Files/tpcds_sf1',
    save_results=True,
    result_table_uri='abfss://......../Tables/dbo/results'
    )
benchmark.run(mode="power_test")

## Run TPCDS `query` test: q1 run 4 times

In [ ]:
from lakebench.engines import HDISpark
from lakebench.benchmarks import TPCDS

engine = HDISpark(
    schema_name = 'spark_tpcds_sf1',
    spark_measure_telemetry = False
)

benchmark = TPCDS(
    engine=engine,
    scenario_name="SF1 - Q4*4",
    input_parquet_folder_uri='abfss://........./Files/tpcds/sf1_parquet',
    save_results=True,
    result_table_uri='abfss://......../Tables/dbo/results',
    query_list=['q1'] * 4
    )
benchmark.run(mode="query")

## Run TPCH Query Test (Run all queries)

In [ ]:
from lakebench.engines import HDISpark
from lakebench.benchmarks import TPCH

engine = HDISpark(
    schema_name = 'spark_tpch_sf10',
    spark_measure_telemetry = False
)

benchmark = TPCH(
    engine=engine,
    scenario_name="SF10 - All Queries",
    input_parquet_folder_uri='abfss://........./Files/tpcds/sf10_parquet',
    save_results=True,
    result_table_uri='abfss://......../Tables/dbo/results'
    )
benchmark.run(mode="query")